In [2]:
from manim import *
import numpy as np
from scipy.integrate import solve_ivp
import math

Manim Community v0.17.3

## Intro

This is based on the example from the 3blue1brown youtube video, adjusted to use the public version of manim.

These key differences exist because of using the community version of manim instead of the 3blue1brown version:

  1. no glow dots - only 3D dots
  1. traced paths don't seem as smooth in the community version as his (or as accurate)
  1. The latex stuff doesn't seem to work at all like in his example
  1. Minor things like `set_backstroke()` that he calls that don't exist, and I can't find info on what they're meant to do

## The Math Part

This is a dependency of __all cells below__.

This was mostly generated by __ChatGPT__ with some slight tweaking by me.  It may not exactly match the 3blue1brown version, though he said he used ChatGPT as well.

In [3]:
def lorenz_attractor(sigma=10.0, rho=28.0, beta=8.0/3.0, 
                     initial_values=[1.0, 1.0, 1.0], 
                     duration=50, resolution=100):
    """
    Generates points representing a path of the Lorenz attractor.
    
    Parameters:
    - sigma, rho, beta: float, parameters for the Lorenz system.
    - initial_values: list of float, initial values [x0, y0, z0].
    - duration: float, total time in seconds for integration.
    - resolution: int, number of points per second.
    
    Returns:
    - points: np.ndarray of shape (duration * resolution, 3), the Lorenz attractor path.
    """
    
    def lorenz_system(t, state):
        x, y, z = state
        dxdt = sigma * (y - x)
        dydt = x * (rho - z) - y
        dzdt = x * y - beta * z
        return [dxdt, dydt, dzdt]

    # Calculate the total number of points and create the time array
    total_points = int(duration * resolution)
    t_eval = np.linspace(0, duration, total_points)
    
    # Solve the Lorenz system with the initial conditions
    sol = solve_ivp(lorenz_system, [0, duration], initial_values, t_eval=t_eval)
    
    # Stack the solution in the form of a 2D array with shape (total_points, 3)
    points = np.vstack((sol.y[0], sol.y[1], sol.y[2])).T
    
    return points

## Testing the Math Part

In [7]:
print(lorenz_attractor(initial_values=[1, 1, 1]))
print(lorenz_attractor(initial_values=[1, 1, 1.01])) # slightly different to demonstrate chaotic motion

[[ 1.          1.          1.        ]
 [ 1.01250667  1.26005694  0.98488796]
 [ 1.04876652  1.52420399  0.97311353]
 ...
 [-0.83922835 -1.06862977 15.54840233]
 [-0.86576271 -1.16522131 15.14857171]
 [-0.89919175 -1.26814793 14.76046073]]
[[ 1.          1.          1.01      ]
 [ 1.01250189  1.25995784  0.99462426]
 [ 1.04874793  1.52400458  0.98259189]
 ...
 [ 1.46203039  1.95813604 16.13120276]
 [ 1.51687142  2.11744842 15.73657906]
 [ 1.58220731  2.28836728 15.35607479]]


## The Scene

In [15]:
%%manim -v WARNING -qm Attractor1

class Attractor1(ThreeDScene):
    def construct(self):
        run_time = 30
        
        # Set up axes
        axes = ThreeDAxes(
            x_range=(-50, 50, 5),
            y_range=(-50, 50, 5),
            z_range=(-0, 50, 5),
            x_length=16,
            y_length=16,
            z_length=8,
        )
        axes.set(width=8)
        axes.center()
        self.add(axes)
    
        # Move the camera
        self.set_camera_orientation(theta=math.radians(43), phi=math.radians(76), gamma=math.radians(1), frame_center=IN)
        # Missing: height=10 orientation param from his example (was it trying to zoom?  not sure - looks ok with it)
    
        # Add the equations as a camera overlay
        # The video colorizes x, y, and z separately
        # I tried the way given and many alternatives and could not get it to work.
        # For example:
        #   - set_color_from_tex() and set_color_from_tex_map()
        #   - various internal operators like \textcolor{}
        #   - breaking the text with {}, {{}}, and the Tex() param to break sections
        equations = Tex(
            R"""
            \[
            \begin{aligned}
            \frac{\mathrm{d} x}{\mathrm{~d} t} & =\sigma(y-x) \\
            \frac{\mathrm{d} y}{\mathrm{~d} t} & = x(\rho-z)-y \\
            \frac{\mathrm{d} z}{\mathrm{~d} t} & = x y-\beta z
            \end{aligned}
            \]
            """,
            font_size=30,
        )
        equations.to_edge(UL)
        #equations.set_backstroke()  # that doesn't exist in community edition (not clear what supposed to do)
        self.add_fixed_in_frame_mobjects(equations)
        self.play(Write(equations))
    
        # Get sets of points that differ slightly in initial conditions
        # to demonstrate chaotic motion.
        epsilon=1e-5
        paths = [lorenz_attractor(initial_values=[10, 10, 10+n*epsilon], duration=run_time) for n in range(10)]
        
        # Translate from axis coordinates to scene points
        points = [axes.coords_to_point(coords) for coords in paths]
        
        # Create curves based on the 2 paths.
        curves = [VMobject().set_points_smoothly(subpoints) for subpoints in points]
        curves_group = VGroup(*curves)  # optimization (according to the video)
        
        # Style the curves
        # This is here as an example but is moot because of the tracing tails.
        # (the curve opacities get set to zero later)
        colors = color_gradient([BLUE_E, BLUE_A], len(curves))
        for i in range(len(colors)):
            curves[i].set_stroke(color=colors[i], opacity=0.25, width=1) # specifying width and opacity moot here
        curves_group.set_stroke(width=2, opacity=1)
        
        # Add dots at the curve heads
        dots = [Dot3D(color=colors[i], radius=0.1) for i in range(len(colors))]
        dots_group = Group(*dots)
        def update_dots(dots_group):
            for dot, curve in zip(dots, curves):
                dot.move_to(curve.get_end())
        dots_group.add_updater(update_dots)
        self.add(dots_group)
        
        # Add tracing tails
        tails = [TracedPath(curves[i].get_end, dissipating_time=3, stroke_color=colors[i], stroke_width=2, stroke_opacity=[0, 1]) for i in range(len(curves))]
        tails_group = VGroup(*tails)
        self.add(tails_group)
        curves_group.set_opacity(0)  # hide the curves so we see the tracing tails instead
        # NOTE: at one point in the video the trails disappears while the dots keep moving
        #       I have no idea what that's about - maybe the tails fns aren't getting called enough.
        
        # Animate the curve creation and camera movement
        self.move_camera(theta = math.radians(43) + run_time*math.radians(3), run_time=run_time, rate_func=linear, added_anims=[
            Create(curve) for curve in curves
        ])

## No Tails Scene

This is just copied and modified to use the real curves instead of the tails.  I should have modularized it instead of copying and pasting, but I didn't feel like re-rendering again after doing it so many times during the debugging.

In [16]:
%%manim -v WARNING -qm Attractor2

class Attractor2(ThreeDScene):
    def construct(self):
        run_time = 30
        
        # Set up axes
        axes = ThreeDAxes(
            x_range=(-50, 50, 5),
            y_range=(-50, 50, 5),
            z_range=(-0, 50, 5),
            x_length=16,
            y_length=16,
            z_length=8,
        )
        axes.set(width=8)
        axes.center()
        self.add(axes)
        
        # Move the camera
        self.set_camera_orientation(theta=math.radians(43), phi=math.radians(76), gamma=math.radians(1), frame_center=IN)
        # Missing: height=10 orientation param from his example (was it trying to zoom?  not sure - looks ok with it)
    
        # Add the equations as a camera overlay
        # The video colorizes x, y, and z separately
        # I tried the way given and many alternatives and could not get it to work.
        # For example:
        #   - set_color_from_tex() and set_color_from_tex_map()
        #   - various internal operators like \textcolor{}
        #   - breaking the text with {}, {{}}, and the Tex() param to break sections
        equations = Tex(
            R"""
            \[
            \begin{aligned}
            \frac{\mathrm{d} x}{\mathrm{~d} t} & =\sigma(y-x) \\
            \frac{\mathrm{d} y}{\mathrm{~d} t} & = x(\rho-z)-y \\
            \frac{\mathrm{d} z}{\mathrm{~d} t} & = x y-\beta z
            \end{aligned}
            \]
            """,
            font_size=30,
        )
        equations.to_edge(UL)
        #equations.set_backstroke()
        self.add_fixed_in_frame_mobjects(equations)
        self.play(Write(equations))
    
        # Get sets of points that differ slightly in initial conditions
        # to demonstrate chaotic motion.
        epsilon=1e-5
        paths = [lorenz_attractor(initial_values=[10, 10, 10+n*epsilon], duration=run_time) for n in range(10)]
        
        # Translate from axis coordinates to scene points
        points = [axes.coords_to_point(coords) for coords in paths]
        
        # Create curves based on the 2 paths.
        curves = [VMobject().set_points_smoothly(subpoints) for subpoints in points]
        curves_group = VGroup(*curves)  # optimization (according to the video)
        
        # Style the curves
        colors = color_gradient([BLUE_E, BLUE_A], len(curves))
        for i in range(len(colors)):
            curves[i].set_stroke(color=colors[i], opacity=0.25, width=1) # specifying width and opacity moot here
        curves_group.set_stroke(width=2, opacity=1)
        
        # Add dots at the curve heads
        dots = [Dot3D(color=colors[i], radius=0.1) for i in range(len(colors))]
        dots_group = Group(*dots)
        def update_dots(dots_group):
            for dot, curve in zip(dots, curves):
                dot.move_to(curve.get_end())
        dots_group.add_updater(update_dots)
        self.add(dots_group)
        
        # Animate the curve creation and camera movement
        self.move_camera(theta = math.radians(43) + run_time*math.radians(3), run_time=run_time, rate_func=linear, added_anims=[
            Create(curve) for curve in curves
        ])